In [4]:
#Importing libraries

In [1]:
import numpy as np
import cv2
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#Data Preprocessing

In [3]:
#Training Image processing

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 3885 images belonging to 5 classes.


In [5]:
#Test Image processing

In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 4317 images belonging to 5 classes.


In [7]:
#Building Model

In [8]:
cnn = tf.keras.models.Sequential()

In [9]:
#Building Convolution layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [12]:
cnn.add(tf.keras.layers.Dropout(0.5))

In [13]:
cnn.add(tf.keras.layers.Flatten())

In [14]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [15]:
cnn.add(tf.keras.layers.Dense(units=5 , activation='softmax'))

In [16]:
cnn.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [21]:
cnn.fit(x = training_set , validation_data = test_set , epochs = 30)

Epoch 1/30
122/122 [==============================] - 23s 185ms/step - loss: 1.3488 - accuracy: 0.4126 - val_loss: 1.0910 - val_accuracy: 0.5705
Epoch 2/30
122/122 [==============================] - 20s 161ms/step - loss: 1.1053 - accuracy: 0.5634 - val_loss: 0.9989 - val_accuracy: 0.6159
Epoch 3/30
122/122 [==============================] - 20s 166ms/step - loss: 0.9947 - accuracy: 0.6147 - val_loss: 0.8524 - val_accuracy: 0.6704
Epoch 4/30
122/122 [==============================] - 21s 170ms/step - loss: 0.9174 - accuracy: 0.6391 - val_loss: 0.8159 - val_accuracy: 0.6789
Epoch 5/30
122/122 [==============================] - 21s 169ms/step - loss: 0.8661 - accuracy: 0.6615 - val_loss: 0.7978 - val_accuracy: 0.6956
Epoch 6/30
122/122 [==============================] - 20s 168ms/step - loss: 0.8146 - accuracy: 0.6875 - val_loss: 0.8279 - val_accuracy: 0.6891
Epoch 7/30
122/122 [==============================] - 20s 168ms/step - loss: 0.7941 - accuracy: 0.6932 - val_loss: 0.6653 - val_ac

In [29]:
#Preprocess New Image

In [30]:
from keras.preprocessing import image
test_image = image.load_img('Prediction/tulip1.jpg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
training_set.class_indices

1/1 [==============================] - 0s 17ms/step


{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [31]:
print(result)

[[0. 0. 0. 0. 1.]]


In [28]:
if result[0][0]==1:
    print('Daisy')
elif result[0][1]==1:
    print('Dandelion')
elif result[0][2]==1:
    print('Rose')
elif result[0][3]==1:
    print('SunFlower')
elif result[0][4]==1:
    print("Tulip")

Tulip


In [17]:
# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera, adjust as needed

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    frame = cv2.resize(frame, (64, 64))
    frame = frame / 255.0  # Normalize the pixel values
    frame = np.expand_dims(frame, axis=0)

    # Use your model to predict the class of the frame
    result = cnn.predict(frame)

    # Get the class label
    class_index = np.argmax(result)
    class_label = list(training_set.class_indices.keys())[class_index]

    # Display the class label on the frame
    cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
